# Derived wind (produto dos ventos)

## Para que serve?

Código voltado para o tratamento do produto dos ventos (ABI Derived Motion Wind), que possui o formato NetCDF (nc). É feita uma filtragem dos dados de acordo com a extensão geográfica desejada, resultando em duas imagens: uma png com o plot das informações, utilizando flechinhas em escala que representam a direção e velocidade dos ventos; e uma imagem float (formato exr) em que cada pixel contém as informações de velocidade e direção do vento, utilizando coordenadas polares e apenas armazenando seus valores, será melhor descrito adiante.

## Bibliotecas necessárias:

In [ ]:
# Required modules
from netCDF4 import Dataset                           # Read / Write NetCDF4 files
import matplotlib.pyplot as plt                       # Plotting library
import os                                             # Miscellaneous operating system interfaces
from osgeo import gdal                                # Python bindings for GDAL
import numpy as np                                    # Scientific computing with Python
from mpl_toolkits.basemap import Basemap              # Import the Basemap toolkit 
import math                                           # Import the Math package
import numpy as np
os.environ['OPENCV_IO_ENABLE_OPENEXR'] = 'true'
import cv2
gdal.PushErrorHandler('CPLQuietErrorHandler')         # Ignore GDAL warnings
import imageio

## Detalhando o código...

Primeiramente, define-se algumas variáveis:
- output: diretório em que serão salvas as imagens
- extent: recorte da extensão desejada, deve-se colocar a mínima longitude, mínima latitude, máxima longitude e máxima latitude, respectivamente. Os valores abaixo são do recorte do Brasil
- title: título da imagem png a ser gerada

In [ ]:
# Output directory
output = "Output_DMW"; os.makedirs(output, exist_ok=True)

# Desired data:
extent = [-75.0, -34, -34, 5.5] # Min lon, Min lat, Max lon, Max lat
title = "Derivated_Motion_Winds"

O arquivo nc é aberto, de acordo com o diretório passado abaixo (lembre-se de alterar conforme a pasta em que está o arquivo) e suas variáveis internas são separadas em novas variáveis no código.

In [ ]:
# Opening the NetCDF Derivated Motion Winds
nc = Dataset('C:\\Gaia Senses\\python_goes\\Samples_DMW\\OR_ABI-L2-DMWF-M6C14_G16_s20212810600206_e20212810609514_c20212810623429.nc') 

# Read the required variables: ================================================ 
pressure = nc.variables['pressure'][:]
temperature = nc.variables['temperature'][:]
wind_direction = nc.variables['wind_direction'][:]
wind_speed = nc.variables['wind_speed'][:]
lats = nc.variables['lat'][:]
lons = nc.variables['lon'][:]

Em seguida, é feita uma filtragem, separando os dados que estão dentro do recorte (extent) desejado, comparando se está dentro do intervalo das latitudes e longitudes máximas e mínimas.

In [ ]:
# Selecting data only from the region of interest: ============================
# Detect Latitude lower and upper index, according to the selected extent: 
latli = np.argmin( np.abs( lats - extent[1] ) ) # Lower index
latui = np.argmin( np.abs( lats - extent[3] ) ) # Upper index

# Detect the Longitude index:
# Store the indexes where the lons are between the selected extent:
lon_ind = np.where(( lons >= extent[0]) & (lons <= extent[2] ))[0]
# Eliminate the lon indexes where we don't have the lat indexes:
lon_ind = lon_ind[(lon_ind >= latui) & (lon_ind <= latli)]

Após isso, ocorre uma separação das variáveis em listas e a leitura destas como numpy arrays, utilizando a biblioteca numpy.

In [ ]:
# Create the variables lists ==================================================
pressure_a = []
temperature_a = []
wind_direction_a = []
wind_speed_a = []
lats_a = []
lons_a = []

# For each item, append the values to the respective variables ================
for item in lon_ind:
    lons_a.append(lons[item])
    lats_a.append(lats[item])
    pressure_a.append(pressure[item])
    temperature_a.append(temperature[item])
    wind_direction_a.append(wind_direction[item])
    wind_speed_a.append(wind_speed[item])

# Read the variables as numpy arrays
temperature = np.asarray(temperature_a)
wind_direction = np.asarray(wind_direction_a)
wind_speed = np.asarray(wind_speed_a)
lons = np.asarray(lons_a)
lats = np.asarray(lats_a)

Na parte abaixo, é feita uma checagem do intervalo de pressão e definida uma cor para representar nas flechinhas. No caso, optei por deixar um único intervalo e cor, mas isso pode ser alterado caso necessário. Inclusive fiz alguns exemplos com diversos intervalos, resultando em uma imagem colorida, porém no fim não era algo que valia a pena para este projeto.

In [ ]:
pressure = np.asarray(pressure_a)
pressure_index = np.where(( pressure >= 100 ) & ( pressure <= 1000 ))[0]
color = '#0000FF' # Blue 

Novamente as variáveis são separadas em listas, contendo apenas os dados do intervalo de pressão escolhido, e em seguida lidas como numpy arrays. No caso, como há apenas um intervalo de pressão que engloba tudo, essa parte é um tanto desnecessária e repetitiva, mas para o caso em que existam intervalos diferentes esta deve ser considerada, colocando em um loop externo que passe por todas as cores.

In [ ]:
# Create the variables lists (considerign only the given pressure range)
pressure_b = []
temperature_b = []
wind_direction_b = []
wind_speed_b = []
lats_b = []
lons_b = []

# For each item, append the values to the respective variables 
for item in pressure_index:
    lons_b.append(lons_a[item])
    lats_b.append(lats_a[item])
    pressure_b.append(pressure_a[item])
    temperature_b.append(temperature_a[item])
    wind_direction_b.append(wind_direction_a[item])
    wind_speed_b.append(wind_speed_a[item])
    
# Final variables for the given pressure range
# Read the variables as numpy arrays
pressure = np.asarray(pressure_b)
temperature = np.asarray(temperature_b)
wind_direction = np.asarray(wind_direction_b)
wind_speed = np.asarray(wind_speed_b)
lons = np.asarray(lons_b)
lats = np.asarray(lats_b)

Aqui são criadas 4 listas. Duas delas são para armazenar as componentes u e v baseadas na velocidade e direção do vento, a partir dessas listas será feito o plot das flechinhas na imagem (conforme será visto adiante). Repare que todas as listas de variáveis criadas anteriormente possuem a mesma ordem, ou seja, a velocidade em wind_speed[0] corresponde a longitude lons[0] e a latitude lats[0], por exemplo. Logo, para cada item é feita a conta dos componentes u e v, baseado na seguinte [referência](https://earthscience.stackexchange.com/questions/11982/plotting-wind-barbs-in-python-no-u-v-component).

Já as listas componente_x_float e componente_y_float são para gerar a imagem exr. Primeiramente, note que os dados são semelhantes a coordenadas polares, em que a velocidade (wind_speed) é o r e a direção (wind_direction) é o theta. Com isso, podemos converter esses valores para coordenadas retangulares, ficando com x e y, que são armanezados nessas listas. Lembrando que isso apenas é feito para dados não nulos (diferentes de 'nan'), caso contrário o valor zero é atribuído automaticamente.

In [ ]:
# Calculating the u and v components using the wind_speed and wind direction
u = []
v = []

componente_x_float = []
componente_y_float = []
for item in range(lons.shape[0]):
    u.append(-(wind_speed[item]) * math.sin((math.pi / 180) * wind_direction[item])) 
    v.append(-(wind_speed[item]) * math.cos((math.pi / 180) * wind_direction[item]))
    
    if str(wind_speed[item]) != 'nan':
        aux_x = wind_speed[item] * math.cos(wind_direction[item]) # Polar to rectangular coordinates
        componente_x_float.append(aux_x)
        aux_y = wind_speed[item] * math.sin(wind_direction[item]) # Polar to rectangular coordinates
        componente_y_float.append(aux_y)
    else:
        componente_x_float.append(0)
        componente_y_float.append(0)

Há uma imagem float de base (que criei a parte) que possui todas as posições zeradas. A ideia é alterar essa imagem base, mudando os valores das posições que possuem algum dado de vento, de acordo com a lista de longitudes e latitudes. Dessa forma, onde está acontecendo algo terá um dado, enquanto nos demais locais o valor será 0. Como há mais posições que não acontecem nada do que posições em que acontecem, essa foi uma maneira de otimizar a produção da imagem. 

A imagem que utilizei possui uma dimensão padrão de 403 x 389, caso queira diferente se atente a mudar essa informação nas contas. Então, temos que cada posição geográfica (latitude, longitude) dentro do recorte possui uma posição equivalente na imagem 403 x 389, logo é feita uma conta de escala para encontrar essa posição correspondente. Isso é feito para cada valor de latitude e longitude, modificando tal posição da imagem com os valores de x e y (mencionados no parágrafo anterior). É importante mencionar que pixel_x e pixel_y são invertidos na hora de atribuir os valores da imagem (vide código), eu não sei porquê isso acontece, mas segundo diversos testes que realizei, essa é a forma que o resultado fica coerente. Resumindo, na imagem exr o valor do pixel x é o valor de x da coordenada retangular e o valor do pixel y é o valor de y da coordenada retangular, o pixel z é zerado.

Com isso, a imagem exr é gerada utilizando a biblioteca imageio.

In [ ]:
img = cv2.imread("FloatBase.exr") # Base with all positions zeroed
img = img.astype(np.float32)
for i in range(lons.size):
    pixel_x = int(((lons[i] + 75) * 403)/41) # Pixel position that refers to a certain longitude (scale)
    if 5.5 >= lats[i] >= -34: # If it is within the latitude range
        pixel_y = 389 - int(((lats[i] + 34) * 389)/39.5) # Pixel position that refers to a certain latitude (scale)
        if pixel_x < 403 and pixel_y < 389: # If it is within the dimension of the image
            img[pixel_y, pixel_x] = [componente_x_float[i], componente_y_float[i], 0] # pixel_x e pixel_y are inverted, don't know why

imageio.imwrite("FloatImageDMW6.exr", img)

Agora o foco é na imagem png, utilizando as componentes u e v, calculadas anteriormente, a função externa Basemap e os valores de longitudes e latitudes das listas, é feito o plot da imagem e das flechinhas de vento.

In [ ]:
# Read the u and v components as numpy arrays
u_comp = np.asarray(u) 
v_comp = np.asarray(v)
bmap = Basemap(llcrnrlon=extent[0], llcrnrlat=extent[1], urcrnrlon=extent[2], urcrnrlat=extent[3], epsg=4326)
x,y = bmap(lons, lats)
bmap.barbs(x, y, u_comp, v_comp, length=2, pivot='middle', barbcolor=color) # Placing the barbs in the image to be plotted

plt.axis('off')

Por fim, a imagem png é salva e mostrada na tela.

In [ ]:
# Save the image
plt.savefig(f'C:\\Gaia Senses\\python_goes\\{output}\\{title}{i}.png', transparent=True, bbox_inches='tight')

# Show the image
plt.show()

Cabe ressaltar que o código foi feito com a ajuda do curso “Processamento de Dados de Satélites Geoestacionários com Python” fornecido pelo INPE (Instituto Nacional de Pesquisas Espaciais).